In [1]:
# Imoort Libraries
import os
import pandas as pd
import seaborn as sns
import env
import wrangle
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from datetime import datetime
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# ACQUIRE

- Data acquired via https://www.kaggle.com/datasets/sherrytp/airline-delay-analysis 
- Data set consisted of 10 csv files
- AA data set has 10 columns and 7628937 rows
### DATA TYPE SUMMARY
- 1 integer
- 3 object 
- 5 float
- 1 datetime value


### MASK FOR AMERICAN AIRLINES 

In [2]:
# Using the 'wrangle' module, this line pulls airline data for the airline code 'AA' (American Airlines) and stores it in the 'flights' variable.
flights = wrangle.pull_airline_data('AA')

In [3]:
# This line of code counts the number of occurrences for each unique carrier (OP_CARRIER) in the 'flights' dataset. 
# It will return a Series with the carriers as the index and the counts as the values, sorted in descending order by default.

flights.OP_CARRIER.value_counts()

AA    7628937
Name: OP_CARRIER, dtype: int64

In [4]:
flights

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
12005,2009-01-01,AA,1,JFK,LAX,0.0,0.0,0.0,0.0,0.0
12006,2009-01-01,AA,2,LAX,JFK,35.0,0.0,0.0,0.0,0.0
12007,2009-01-01,AA,3,JFK,LAX,0.0,0.0,0.0,0.0,0.0
12008,2009-01-01,AA,5,DFW,HNL,0.0,0.0,0.0,0.0,0.0
12009,2009-01-01,AA,6,OGG,DFW,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7419921,2019-12-31,AA,1403,BWI,DFW,0.0,0.0,0.0,0.0,0.0
7419922,2019-12-31,AA,1404,DEN,PHX,0.0,0.0,0.0,0.0,0.0
7419923,2019-12-31,AA,1405,DFW,LAX,35.0,0.0,0.0,0.0,0.0
7419924,2019-12-31,AA,1406,DFW,SAT,0.0,0.0,0.0,0.0,0.0


 # PREPARE
 
- Sorting columns
- Dropping duplicate rows
- Dropping missing values
- Renaming columns
- Resetting index
- Type conversion
- Filtering
- Aggregation

In [18]:
# Using the 'wrangle' module, clean the 'flights' dataframe to prepare it for computing the average daily delay.
df = wrangle.clean_flight_data_for_average_daily_delay(flights)


In [19]:
# List of columns representing different types of flight delays to be summed up for analysis.
col_to_sum = ['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']


In [20]:
# List of top 15 busiest airport hubs in the U.S. by airport code

top_15_hubs = ['ATL',  # Hartsfield-Jackson Atlanta International Airport
               'DFW',  # Dallas/Fort Worth International Airport
               'DEN',  # Denver International Airport
               'ORD',  # O'Hare International Airport, Chicago
               'LAX',  # Los Angeles International Airport
               'JFK',  # John F. Kennedy International Airport, New York
               'IAH',  # George Bush Intercontinental Airport, Houston
               'PHX',  # Phoenix Sky Harbor International Airport
               'EWR',  # Newark Liberty International Airport
               'SFO',  # San Francisco International Airport
               'SEA',  # Seattle-Tacoma International Airport
               'IAD',  # Washington Dulles International Airport
               'PHL',  # Philadelphia International Airport
               'CLT',  # Charlotte Douglas International Airport
               'MIA']  # Miami International Airport


In [21]:
# Display the first five rows of the 'flights' DataFrame to get an initial overview of its structure and data.
flights.head().T



,12005,12006,12007,12008,12009
FL_DATE,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00
OP_CARRIER,AA,AA,AA,AA,AA
OP_CARRIER_FL_NUM,1,2,3,5,6
ORIGIN,JFK,LAX,JFK,DFW,OGG
DEST,LAX,JFK,LAX,HNL,DFW
CARRIER_DELAY,0.0,35.0,0.0,0.0,0.0
WEATHER_DELAY,0.0,0.0,0.0,0.0,0.0
NAS_DELAY,0.0,0.0,0.0,0.0,0.0
SECURITY_DELAY,0.0,0.0,0.0,0.0,0.0
LATE_AIRCRAFT_DELAY,0.0,0.0,0.0,0.0,0.0


### OBSERVED NUMERICAL FEATURES

In [22]:
# Retrieve the number of rows and columns in the 'flights' DataFrame.
flights.shape


(7628937, 10)

In [34]:
# Observe data types

flights.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7628937 entries, 12005 to 7419925
Data columns (total 10 columns):
 #   Column               Dtype         
---  ------               -----         
 0   FL_DATE              datetime64[ns]
 1   OP_CARRIER           object        
 2   OP_CARRIER_FL_NUM    int64         
 3   ORIGIN               object        
 4   DEST                 object        
 5   CARRIER_DELAY        float64       
 6   WEATHER_DELAY        float64       
 7   NAS_DELAY            float64       
 8   SECURITY_DELAY       float64       
 9   LATE_AIRCRAFT_DELAY  float64       
dtypes: datetime64[ns](1), float64(5), int64(1), object(3)
memory usage: 640.2+ MB


In [12]:
# Iterate over each column in the DataFrame
for col in df.columns:
    # Print column name, a separator, and then the value counts for that column
    print(f"\n{col}:\n{'-'*20}\n{df[col].value_counts()}\n")



average_delay:
--------------------
10.470068    31
17.975713    31
9.139625      1
24.000000     1
6.464414      1
             ..
9.833932      1
13.022064     1
14.819889     1
17.791667     1
9.269166      1
Name: average_delay, Length: 3957, dtype: int64



In [26]:
# Display the first 5 rows of the dataframe 'df' to get an overview of its structure and data.
df.head()


,average_delay
FL_DATE,
2009-01-01,9.139625
2009-01-02,9.778761
2009-01-03,12.781872
2009-01-04,9.834304
2009-01-05,35.406053


In [27]:
# Display the last 5 rows of the dataframe 'df' to get an overview of its structure and data.

df.tail()

,average_delay
FL_DATE,
2019-12-27,11.728281
2019-12-28,19.339775
2019-12-29,12.237557
2019-12-30,12.854688
2019-12-31,9.269166


In [28]:
# Generate a statistical summary of the 'flights' dataframe, 
# providing an overview of the distribution for each numerical column.

flights.describe().T


,count,mean,std,min,25%,50%,75%,max
OP_CARRIER_FL_NUM,7628937.0,1357.323455,746.459565,1.0,678.0,1386.0,1951.0,2949.0
CARRIER_DELAY,7628937.0,3.971183,27.446161,0.0,0.0,0.0,0.0,2402.0
WEATHER_DELAY,7628937.0,0.655725,9.528406,0.0,0.0,0.0,0.0,1934.0
NAS_DELAY,7628937.0,2.856393,13.294111,0.0,0.0,0.0,0.0,1439.0
SECURITY_DELAY,7628937.0,0.017055,1.309241,0.0,0.0,0.0,0.0,1078.0
LATE_AIRCRAFT_DELAY,7628937.0,4.355710,22.630550,0.0,0.0,0.0,0.0,1778.0


In [29]:
# Extracting the frequency counts of each unique airport in the 'ORIGIN' column of the 'flights' dataframe.
# The result is stored in 'list_of_airports' which is then converted into a DataFrame.
# Finally, we're finding the number of unique counts (i.e., the number of times each airport frequency occurs) using 'nunique()'.

list_of_airports = flights['ORIGIN'].value_counts()
list_of_airports = pd.DataFrame(list_of_airports)
list_of_airports.nunique()


ORIGIN    122
dtype: int64

In [32]:
# Display the first 15 rows of the 'list_of_airports' DataFrame to get a quick overview of the top airports by frequency.
list_of_airports.head(15)


,ORIGIN
DFW,1652357
ORD,633252
MIA,529718
CLT,452426
LAX,375569
PHX,293911
PHL,235608
LGA,209935
BOS,193868
DCA,184541


## TRAIN SPLIT

In [35]:
# Splitting the data by year



# EXPLORE

In [ ]:
# Calculate the total delay for each row by summing the delays across the specified columns.

row_sums = flights[['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']].sum(axis=1)
row_sums

In [ ]:
# List of top 15 busiest airport hubs in the U.S. by airport code

top_15_hubs = ['ATL',  # Hartsfield-Jackson Atlanta International Airport
               'DFW',  # Dallas/Fort Worth International Airport
               'DEN',  # Denver International Airport
               'ORD',  # O'Hare International Airport, Chicago
               'LAX',  # Los Angeles International Airport
               'JFK',  # John F. Kennedy International Airport, New York
               'IAH',  # George Bush Intercontinental Airport, Houston
               'PHX',  # Phoenix Sky Harbor International Airport
               'EWR',  # Newark Liberty International Airport
               'SFO',  # San Francisco International Airport
               'SEA',  # Seattle-Tacoma International Airport
               'IAD',  # Washington Dulles International Airport
               'PHL',  # Philadelphia International Airport
               'CLT',  # Charlotte Douglas International Airport
               'MIA']  # Miami International Airport


In [ ]:
# Display the first 15 rows of the 'list_of_airports' dataframe to get a quick overview of its data structure and contents.
list_of_airports.head(15)


### Investigate the relationship between these columns and determine methods for merging them.

### Evaluate the columns using different criteria and metrics.


In [ ]:
#Fills in nulls as zero as null means no delay
flights.fillna(0, inplace=True)
#Makes FL_DATE column a datetime datatype
flights.FL_DATE = flights.FL_DATE.astype('datetime64')
#Makes FL_DATE the index
# flights = flights.set_index('FL_DATE')

In [ ]:
flights_cat = flights
flights_cat['carrier'] = flights_cat.CARRIER_DELAY > 0

In [ ]:
flights_cat['weather'] = flights_cat.WEATHER_DELAY > 0

In [ ]:
flights_cat['nas'] = flights_cat.NAS_DELAY > 0

In [ ]:
flights_cat['security'] = flights_cat.SECURITY_DELAY > 0

In [ ]:
flights_cat['late_aircraft'] = flights_cat.LATE_AIRCRAFT_DELAY > 0

In [ ]:
flights_cat.drop(columns=['OP_CARRIER', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY'])

In [ ]:
flights.info()

### At first glance it seems weird that weather does not have as many rows as the rest.  As a pilot I thought that weather dominates the delays so I wondered why this is the case.

### It seems that the NAS data includes non extreme weater as does carrier and late aircraft delay.

In [ ]:
dep = pd.crosstab(flights_cat.FL_DATE, [flights_cat.carrier, flights_cat.weather, flights_cat.nas, flights_cat.security, flights_cat.late_aircraft])

### Chi squared
* Null Hypothesis:  These delay types are independent of each other
* Alternative Hypothesis:  These types are dependent on each other
* Alpha < 0.05

In [ ]:
dep

In [ ]:
chi2, p, degf, expected = stats.chi2_contingency(dep)

In [ ]:
chi2, p

#### There is sufficient evidence to reject the null.  They are dependent on each other.

#### This suggests that there may bee some sort of overlap.  A quick look at the NAS delays shows that minor weather phenomena show up in the NAS as opposed to weather.  Also a weather delay on one leg of an airplanes trip seems to turn into a carrier delay on the next.

### A look at histograms

In [ ]:
plt.hist(flights.CARRIER_DELAY, bins=100)

In [ ]:
plt.hist(flights.WEATHER_DELAY, bins=100)

In [ ]:
plt.hist(flights.NAS_DELAY, bins=100)

In [ ]:
plt.hist(flights.SECURITY_DELAY, bins=100)

In [ ]:
plt.hist(flights.LATE_AIRCRAFT_DELAY, bins=100)

#### Skewed makes a whole lot of sense.  Nothing unexpected here.

### THERE IS sufficient evidence that I can combine these columns in feature engineering.

### Resample my data by daily mean delays

In [ ]:
flights_daily_mean = wrangle.clean_flight_data_for_average_daily_delay(flights)

In [ ]:
flights_daily_mean.sort_index()

In [ ]:
flights_daily_mean.isna().sum()

### Find the best interval to define average delay by to optimize reducing noise while keeping good correlation results

In [ ]:
def plot_average_by_interval():
    
    resample_dict = {'D':'Daily', 'W':'Weekly', '2W':'FORTNIGHTLY', 'M':'Monthly', '3M':'Quarterly', '6M':'Semi-Annually', 'Y':'Yearly'}

    for _ in resample_dict:
        plt.figure(figsize=(10,6))

        flights_daily_mean.resample(_).mean().average_delay.plot(label='daily')
        # flights_daily_mean.resample('W').mean().average_delay.plot(label='weekly')
        # flights_daily_mean.resample('M').mean().average_delay.plot(label='monthly')
        # flights_daily_mean.resample('Y').mean().average_delay.plot(label='yearly')


        plt.title(f'{resample_dict[_][0:]} Average Delay')
        plt.legend()
        plt.show()

        pd.plotting.autocorrelation_plot(flights_daily_mean.average_delay.resample(_).mean())
        plt.show()

plot_average_by_interval()

### Plot the best lag plot based on the airline.  The best will always be 365

In [ ]:
def plot_best_lag_plot():

    #Plots the best correlated lag
    pd.plotting.lag_plot(flights_daily_mean.resample('W').mean(), lag=365)
    plt.title('lag plot');

plot_best_lag_plot()

### Split the data by year

In [ ]:
train = flights_daily_mean.iloc[:2016] 
test = flights_daily_mean.iloc[2016:] 
train.loc['2009-09-30':]

In [ ]:
test

### Plot the seasonal decompositions for the various time periods

In [ ]:
y = train.average_delay.resample('d').mean()

result = sm.tsa.seasonal_decompose(y)

decomposition = pd.DataFrame({
    'y': result.observed,
    'trend': result.trend,
    'seasonal': result.seasonal,
    'resid': result.resid
})

decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
decomposition[['trend_centered', 'seasonal', 'resid']].plot();

In [ ]:
y = train.average_delay.resample('w').mean()

result = sm.tsa.seasonal_decompose(y)

decomposition = pd.DataFrame({
    'y': result.observed,
    'trend': result.trend,
    'seasonal': result.seasonal,
    'resid': result.resid
})

decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
decomposition[['trend_centered', 'seasonal', 'resid']].plot();

In [ ]:
def seasonal_decomposition_plot(duration):
    y = train.average_delay.resample(duration).mean()

    result = sm.tsa.seasonal_decompose(y)

    decomposition = pd.DataFrame({
        'y': result.observed,
        'trend': result.trend,
        'seasonal': result.seasonal,
        'resid': result.resid
    })

    decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
    decomposition[['trend_centered', 'seasonal', 'resid']].plot();
    
seasonal_decomposition_plot('2w')

In [ ]:
y = train.average_delay.resample('m').mean()

result = sm.tsa.seasonal_decompose(y)

decomposition = pd.DataFrame({
    'y': result.observed,
    'trend': result.trend,
    'seasonal': result.seasonal,
    'resid': result.resid
})

decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
decomposition[['trend_centered', 'seasonal', 'resid']].plot();

In [ ]:
y = train.average_delay.resample('3m').mean()

result = sm.tsa.seasonal_decompose(y)

decomposition = pd.DataFrame({
    'y': result.observed,
    'trend': result.trend,
    'seasonal': result.seasonal,
    'resid': result.resid
})

decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
decomposition[['trend_centered', 'seasonal', 'resid']].plot();

In [ ]:
y = train.average_delay.resample('6m').mean()

result = sm.tsa.seasonal_decompose(y)

decomposition = pd.DataFrame({
    'y': result.observed,
    'trend': result.trend,
    'seasonal': result.seasonal,
    'resid': result.resid
})

decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
decomposition[['trend_centered', 'seasonal', 'resid']].plot();

In [ ]:
y = train.average_delay.resample('y').mean()

result = sm.tsa.seasonal_decompose(y)

decomposition = pd.DataFrame({
    'y': result.observed,
    'trend': result.trend,
    'seasonal': result.seasonal,
    'resid': result.resid
})

decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
decomposition[['trend_centered', 'seasonal', 'resid']].plot();

### Once again the best period to resample by seems to be 2 weeks or a fortnight